In [1]:
import pandas as pd
from xgb2sql import *
import xgboost as xgb
from sklearn.model_selection import train_test_split


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
#数据读取
pd.set_option('display.max_columns', None)#显示所有的列
path  = './train.csv'
train = pd.read_csv(path).fillna(-1)
X = train.loc[:,:'DKLL']
Y = train['label']

x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, test_size=0.2)

In [3]:
X.head()

,id,XINGBIE,CSNY,HYZK,ZHIYE,ZHICHEN,ZHIWU,XUELI,DWJJLX,DWSSHY,GRJCJS,GRZHZT,GRZHYE,GRZHSNJZYE,GRZHDNGJYE,GRYJCE,DWYJCE,DKFFE,DKYE,DKLL
0,0,1,1.038672e+09,90,90,999,0,99,150,12,1737.0,1,3223.515,801.310,837.000,312.00,312.00,175237,154112.935,2.708
1,1,2,5.048928e+08,90,90,999,0,99,110,0,4894.0,1,18055.195,53213.220,1065.200,795.84,795.84,300237,298252.945,2.979
2,2,1,7.361856e+08,90,90,999,0,99,150,9,10297.0,1,27426.600,13963.140,7230.020,1444.20,1444.20,150237,147339.130,2.708
3,3,1,4.285152e+08,90,90,999,0,99,150,7,10071.5,1,111871.130,99701.265,2271.295,1417.14,1417.14,350237,300653.780,2.708
4,4,2,5.442048e+08,90,90,999,0,99,900,14,2007.0,1,237.000,11028.875,35.780,325.50,325.50,150237,145185.010,2.708


In [4]:
tree_nums=20

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, test_size=0.2)

params = {
    'eta': 0.1,
    'max_depth': 2,
    'min_child_weight': 3,
    'gamma': 0,
    'subsample': 0.8,
    'objective': 'binary:logistic',
    'eval_metric': 'auc'
}

dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test, y_test)
xgbclassifer = xgb.train(params=params,
                         dtrain=dtrain,
                         num_boost_round=tree_nums,
                         early_stopping_rounds=5,
                         evals=[(dtrain, 'train'), (dtest, 'test')])
x_test = x_test.reset_index(drop=True)
x_test.loc[:,'pred'] = xgbclassifer.predict(dtest)

[0]	train-auc:0.72613	test-auc:0.73329
[1]	train-auc:0.72613	test-auc:0.73329
[2]	train-auc:0.75045	test-auc:0.75460
[3]	train-auc:0.75045	test-auc:0.75460
[4]	train-auc:0.81356	test-auc:0.82227
[5]	train-auc:0.81245	test-auc:0.82053
[6]	train-auc:0.81182	test-auc:0.82116
[7]	train-auc:0.81188	test-auc:0.82100
[8]	train-auc:0.81475	test-auc:0.82306
[9]	train-auc:0.82966	test-auc:0.83087
[10]	train-auc:0.83427	test-auc:0.83489
[11]	train-auc:0.83253	test-auc:0.83428
[12]	train-auc:0.83236	test-auc:0.83359
[13]	train-auc:0.85147	test-auc:0.85260
[14]	train-auc:0.85138	test-auc:0.85253
[15]	train-auc:0.85052	test-auc:0.85369
[16]	train-auc:0.85122	test-auc:0.85221
[17]	train-auc:0.85185	test-auc:0.85461
[18]	train-auc:0.85243	test-auc:0.85527
[19]	train-auc:0.85676	test-auc:0.86080


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:200: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [6]:
#把XGB模型输出为json格式
xgb_json = xgbclassifer.get_dump(dump_format='json')
with codecs.open(r'./result/xgb_binary.json', 'w', encoding="utf-8") as f:
    for single_json in xgb_json:
        single_json = single_json.replace('\n', ' ').replace('\r', ' ')
        f.write(single_json + '\n')

In [7]:
#将json模型文件解析为sql文件
with open(r'./result/xgb_binary.json', 'r') as f_read:
    xgb_json = f_read.readlines()
tree_sqls = parse_xgb_trees(xgb_json)
final_sqls = ''
for item_sql in tree_sqls:
    final_sqls = final_sqls + item_sql

In [8]:
#生成SQL逻辑
with codecs.open(r'./result/xgb_binary_model.sql', 'w', encoding="utf-8") as f:
    for item_sql in tree_sqls:
        f.write(item_sql + '\n') 

In [9]:
#生成树的列表
with codecs.open(r'./result/scores.sql', 'w', encoding="utf-8") as f:
    for i in range(tree_nums):
        f.write('tree_{}_score+'.format(i)+ '\n')

In [11]:
import numpy as np

可以选取几个客户的预测结果和sql生成的结果做对比

In [22]:
x_test.loc[:,'pred'].head()

0    0.085718
1    0.255263
2    0.091822
3    0.553855
4    0.133879
Name: pred, dtype: float32